# Import Libararies

In [29]:
import os
from dotenv import load_dotenv
from langchain_openai import OpenAI , ChatOpenAI , OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate , MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.documents import Document
from langchain.chains import create_retrieval_chain , create_history_aware_retriever
from langchain_core.messages import HumanMessage , AIMessage

In [2]:
load_dotenv()

True

## Intialize OpenAI

In [3]:
    OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [4]:
llm = ChatOpenAI(api_key=OPENAI_API_KEY)

In [5]:
prompt = ChatPromptTemplate.from_messages([
    ("system","You are a English to Hindi translater expert that return whatever user say in Hindi"),
    ("user","{input}")
])

In [6]:
output_parser = StrOutputParser()

chain = prompt | llm | output_parser

chain.invoke({
    "input":"i love programming in python"
})

'मुझे पाइथन में प्रोग्रामिंग करना पसंद है।'

In [7]:
loader = WebBaseLoader("https://iampathak2702.github.io/Resume/")

docs = loader.load()
docs

[Document(page_content='\n\n\n\n\nResume Ved Prakash Pathak\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nIâ€™m Ved Prakash Pathak\nMachine Learning Engg | AI\n\t\t\t\t\t\t\t\t\t\tDeveloper\n\n\t\t\t\t\t\t\t\t\tI am a dynamic professional with a robust background in machine learning and\n\t\t\t\t\t\t\t\t\tartificial intelligence. Driven by curiosity and a pursuit of excellence, I\n\t\t\t\t\t\t\t\t\tspecialize in Python, focusing on NLP, Computer Vision, and Generative AI. With\n\t\t\t\t\t\t\t\t\ta strong work ethic and innovative mindset, I thrive in challenging environments,\n\t\t\t\t\t\t\t\t\tleveraging strategic thinking to drive transformative change. Eager to make a\n\t\t\t\t\t\t\t\t\tsignificant impact in the technology sector, I am ready to seize new opportunities\n\t\t\t\t\t\t\t\t\tand collaborate for remarkable results.\n\t\t\t\t\t\t\t\t\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n Toggle navigation   \n\n\n\n\nHome\nAbout Me\nProjects\nCertificates\nContact\n\n\n\n\n\n\n\n\n\n\

In [8]:
text_splitter = RecursiveCharacterTextSplitter()

documents = text_splitter.split_documents(docs)
documents

[Document(page_content='Resume Ved Prakash Pathak\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nIâ€™m Ved Prakash Pathak\nMachine Learning Engg | AI\n\t\t\t\t\t\t\t\t\t\tDeveloper\n\n\t\t\t\t\t\t\t\t\tI am a dynamic professional with a robust background in machine learning and\n\t\t\t\t\t\t\t\t\tartificial intelligence. Driven by curiosity and a pursuit of excellence, I\n\t\t\t\t\t\t\t\t\tspecialize in Python, focusing on NLP, Computer Vision, and Generative AI. With\n\t\t\t\t\t\t\t\t\ta strong work ethic and innovative mindset, I thrive in challenging environments,\n\t\t\t\t\t\t\t\t\tleveraging strategic thinking to drive transformative change. Eager to make a\n\t\t\t\t\t\t\t\t\tsignificant impact in the technology sector, I am ready to seize new opportunities\n\t\t\t\t\t\t\t\t\tand collaborate for remarkable results.\n\t\t\t\t\t\t\t\t\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n Toggle navigation   \n\n\n\n\nHome\nAbout Me\nProjects\nCertificates\nContact\n\n\n\n\n\n\n\n\n\n\n\nAbout M

In [11]:
embeddings = OpenAIEmbeddings()

vectorstore = FAISS.from_documents(documents=documents, embedding=embeddings)

In [12]:
vectorstore

In [15]:
template = """Answer the following question based on only on the provided context:

<context>
{context}
<context>

Question : {input}

"""

prompt = ChatPromptTemplate.from_template(template)
document_chain = create_stuff_documents_chain(llm, prompt)

In [19]:
document_chain.invoke({
    "input":"Who are we talking about?",
    "context":[Document(page_content="Ved Prakash Pathak a Machine Learning Enginner")]
})

'We are talking about Ved Prakash Pathak, a Machine Learning Engineer.'

In [20]:
retriever = vectorstore.as_retriever()
retrievel_chain = create_retrieval_chain(retriever , document_chain)

In [22]:
response = retrievel_chain.invoke({
    "input":"Who are we talking about?",
})

AttributeError: 'dict' object has no attribute 'answer'

In [24]:
print(response['answer'])

We are talking about Ved Prakash Pathak.


# Conversation Retrival Chain

In [28]:
prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user","{input}"),
    ("user", "Given the above conversation, generate a search query to look up in the order to get information relevant to the conversation")
])

retrievel_chain = create_history_aware_retriever(llm , retriever=retriever , prompt=prompt)

In [30]:
chat_history = [
    HumanMessage(content="Who are we talking about?"),
    AIMessage(content="Yes")
]

retrievel_chain.invoke({
    "chat_history":chat_history,
    "input":"Tell me more about it!"
})

[Document(page_content='Data Visualization\n\n\n \n\n\n\n\nCloud Computing\n\n\n \n\n\n\n\nFASTAPI\n\n\n \n\n\n\n\nHTML & CSS\n\n\n \n\n\n\n\njavascript\n\n\n \n\n\n\n\njinja\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nProjects\nIn my recent projects, I\'ve ventured into a diverse array of fields,\n\t\t\t\t\teach representing a hallmark of my expertise and\n\t\t\t\t\tcommitment to excellence. From pioneering a highly adaptable RAG Based OpenAI Chatbot plugin for\n\t\t\t\t\tWordPress, showcasing\n\t\t\t\t\tmy prowess in AI development and user engagement optimization, to architecting an interactive MCQ\n\t\t\t\t\tgenerator with a\n\t\t\t\t\tstunning 90% content extraction accuracy, I\'ve consistently pushed the boundaries of innovation.\n\t\t\t\t\tConcurrently, my foray\n\t\t\t\t\tinto advanced GAN training for image generation, achieving remarkable 98% convergence and delivering\n\t\t\t\t\thigh-quality,\n\t\t\t\t\trealistic fashion images, underscores my proficiency in cutting-edge

In [31]:
prompt = ChatPromptTemplate.from_messages([
    ("system"," Answer the user's content based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user","{input}")
])

document_chain = create_stuff_documents_chain(llm,prompt)

retrieval_chain = create_retrieval_chain(retrievel_chain,document_chain)


In [35]:
response = retrieval_chain.invoke({
    'chat_history': chat_history,
    'input': "Who are we talking about?"
})


In [36]:
print(response["answer"])

We are talking about Ved Prakash Pathak, a Machine Learning Engineer and AI Developer with expertise in Python, NLP, Computer Vision, and Generative AI. He has a strong background in machine learning methodologies and MLOps practices. Ved is actively seeking opportunities to leverage his skills and contribute to innovative projects in the technology sector.


In [38]:
def chat_bot(api_key: str, url: str, user_input: str) -> str:
    """
    Generates a response based on the user input using a combination of history-aware retrieval and document analysis.

    Args:
        api_key (str): The API key for accessing OpenAI services.
        url (str): URL for accessing relevant documents.
        user_input (str): User's input to generate a response.

    Returns:
        str: Response generated based on the user input and context.
    """

    # Initialize ChatOpenAI instance
    chat_ai = ChatOpenAI(api_key=api_key)
    
    # Load documents from the web
    web_loader = WebBaseLoader(url)
    documents = web_loader.load()
    
    # Generate embeddings for the documents
    embeddings = OpenAIEmbeddings()
    text_splitter = RecursiveCharacterTextSplitter()
    split_documents = text_splitter.split_documents(documents)
    vector_store = FAISS.from_documents(documents=split_documents, embedding=embeddings)
    retriever = vector_store.as_retriever()
    
    # Define prompt for history-aware retrieval
    prompt_retrieval = ChatPromptTemplate.from_messages([
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        ("user", "Based on the conversation above, generate a search query to retrieve relevant information.")
    ])
    
    # Create history-aware retriever
    history_aware_retriever = create_history_aware_retriever(chat_ai, retriever=retriever, prompt=prompt_retrieval)
    
    # Initialize chat history
    chat_history = [
        HumanMessage(content="Who are we talking about?"),
        AIMessage(content="Yes")
    ]
    
    # Define prompt for document analysis
    prompt_analysis = ChatPromptTemplate.from_messages([
        ("system", "Answer the user's content based on the context below:\n\n{context}"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}")
    ])
    
    # Create document analysis chain
    document_analysis_chain = create_stuff_documents_chain(chat_ai, prompt_analysis)
    
    # Create retrieval chain
    retrieval_chain = create_retrieval_chain(history_aware_retriever, document_analysis_chain)
    
    # Invoke retrieval chain to generate response
    response = retrieval_chain.invoke({
        'chat_history': chat_history,
        'input': f"{user_input}"
    })
    
    return response['answer']


In [39]:
response = chat_bot(api_key=OPENAI_API_KEY , url="https://iampathak2702.github.io/Resume/" , user_input="who are we talking about?")

In [40]:
response

'We are talking about Ved Prakash Pathak, a Machine Learning Engineer and AI Developer with expertise in various fields such as NLP, Computer Vision, and Generative AI. He has a strong background in Python and is actively seeking opportunities in the domains of Machine Learning and AI development. Ved Prakash Pathak is driven by curiosity, innovation, and a pursuit of excellence in the technology sector.'